# RedisGraph tests
* https://oss.redislabs.com/redisai/
* https://github.com/RedisAI/RedisAI
* https://github.com/RedisAI/redisai-py

# Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from pprint import pprint
from redisai import Client
from redisgraph import Node, Edge, Graph, Path

In [3]:
import hashlib
from hashlib import sha256

## Connecting

Need to get the `gpu-jupyter` and the `redisai` docker containers connected. If run bare, something like:

    soul@guppy:~$ docker network connect gpu-jupyter_default gpu-jupyter 
    soul@guppy:~$ docker network connect gpu-jupyter_default redis-aig
    $ docker network inspect gpu-jupyter_default 
    
Docker has better ways than this.

In [4]:
con = Client(host='172.19.0.2', port=6379)

### Alive?

In [5]:
con.ping()

True

# Utility functions

In [6]:
def tput(a):
    k = sha256(a).hexdigest()[:16]
    con.tensorset(k, a)
    return k

In [7]:
def tget(k):
    a = con.tensorget(k)
    assert k == sha256(a).hexdigest()[:16]
    return a

# RedisGraph

In [8]:
import redis
from redisgraph import Node, Edge, Graph, Path

# The Model
    Investigation -> Experiment -> multiple ResultDAGs
`ResultDAG` is

    (netState, params)-[mutation]->(netState, params)-[mutation ...
                     +-[mutation]->(netstate, params) ...
etc. `mutation` can be a learning trajectory, or an edit.

Perhaps `mutation` can be expressed in python.

Generally the results of experiments are preferred to be reproducible, but they won't always be, when they import entropy.

## Some neural nets

In [9]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NetMaker, NNMEG

In [10]:
mnm = NetMaker(NNMEG)
xor_net = mnm('2x2tx1t')
adc_net = mnm('1x8tx8tx3t')

## ... and training data

In [11]:
xor_training_batch = (np.array([[-0.5, -0.5],
                            [-0.5,  0.5],
                            [ 0.5,  0.5],
                            [ 0.5, -0.5]]),
                  np.array([[-0.5],
                            [ 0.5],
                            [-0.5],
                            [ 0.5]]))

In [12]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

In [13]:
x = np.arange(0, 1, 1.0/(8*1)).reshape(-1,1) # 1 point in each output region
adc_training_batch = (x, vadc(x))

# The graph database

## utilities

In [14]:
def as_dicts(result):
    #h = list(v[1].decode().split('.',1)[1] for v in result.header)
    h = list(v[1].decode() for v in result.header)
    for v in result.result_set:
        yield dict(zip(h, v))

In [15]:
def as_nets_from_dicts(dg):
    for d in dg:
        # FIXME: shorthand is prefixed, adapt
        yield mnm(d['shorthand']).set_state_from_vector(tget(d['sv']))

## An example experiment's DAG

In [16]:
from redis import ResponseError

In [17]:
#con.flushall() # Nuke the entire contents of redis

In [18]:
#con.memory_purge()
#con.memory_stats()

In [19]:
exp_dag = Graph('experiment', con)

In [20]:
exp_dag.delete()

b'Graph removed, internal execution time: 0.021363 milliseconds'

In [21]:
#%%script echo skipping
try:
    exp_dag.delete()
except (NameError, ResponseError):
    pass

### We use `adc_net`

In [22]:
net = adc_net
training_batch = adc_training_batch

In [23]:
net_start_node = Node(label='net',
                properties={'shorthand': net.shorthand,
                            'ksv': tput(net.state_vector()),
                            'loss': net.losses([training_batch])[0],
                           })

In [24]:
exp_dag.add_node(net_start_node)

In [25]:
exp_dag.commit()

In [26]:
query = """MATCH (n:net)
           RETURN n.shorthand, n.ksv"""

In [27]:
result = exp_dag.query(query)

In [28]:
result.pretty_print()

+----------------+------------------+
| b'n.shorthand' |     b'n.ksv'     |
+----------------+------------------+
|   1x8tx8tx3t   | 4fbc1905f81cf791 |
+----------------+------------------+

Cached execution 0.0
internal execution time 0.38686


In [29]:
list(as_dicts(result))

[{'n.shorthand': '1x8tx8tx3t', 'n.ksv': '4fbc1905f81cf791'}]

In [30]:
#list(as_nets_from_dicts(as_dicts(result)))

## Train, recording trajectory

In [31]:
net_node = net_start_node
loss = net.losses([training_batch])[0]
batch_ctr = 0
while loss > 1e-25:
    batch_ctr_at_seg_start = batch_ctr
    losses = []
    etas = []
    deltas = []
    prior_loss = loss
    prior_net_node = net_node
    while loss / prior_loss > 0.7071 and len(deltas) < 10_000:
        if not etas or net.eta != etas[-1][1]:
            etas.append([batch_ctr, net.eta])
        loss = net.learn([training_batch])
        if batch_ctr < 100 or batch_ctr % 100 == 0:
            losses.append([batch_ctr, loss])
            deltas.append([batch_ctr, net.deltas()])
        batch_ctr += 1
    #if losses[-1][0] < (batch_ctr-1):
    #    losses.append([batch_ctr, loss])
    if not deltas or deltas[-1][0] < (batch_ctr-1):
        deltas.append((batch_ctr, net.deltas()))
    net_node = Node(label='net',
                    properties={'shorthand': net.shorthand,
                                'ksv': tput(net.state_vector()),
                                'loss': loss,
                                })
    properties = dict(zip(deltas[0][1]._fields, map(list, (zip(*(v[1] for v in deltas)))))) # RedisGraph has a tuple allergy
    #properties = {}
    properties['batch_points'] = [v[0] for v in deltas]
    properties['etas'] = etas
    properties['batches_this_segment'] = batch_ctr - batch_ctr_at_seg_start
    #pprint(properties)
    #continue
    mutation = Edge(prior_net_node, 'trained', net_node, properties=properties)
    print(f"loss {loss}, batches cumulative {batch_ctr}, this segment {properties['batches_this_segment']}, len(deltas) {len(deltas)}", end=' ')
    print("add_node", end=' ')
    exp_dag.add_node(net_node)
    print("add_edge", end=' ')
    exp_dag.add_edge(mutation)
    print("commit ", end=' ')
    exp_dag.commit()
    print()

loss 1.217063265894283, batches cumulative 25, this segment 25, len(deltas) 25 add_node add_edge commit  
loss 0.8588770681485749, batches cumulative 69, this segment 44, len(deltas) 44 add_node add_edge commit  
loss 0.6067287496652942, batches cumulative 283, this segment 214, len(deltas) 34 add_node add_edge commit  
loss 0.42893965748085416, batches cumulative 533, this segment 250, len(deltas) 4 add_node add_edge commit  
loss 0.3017505935505457, batches cumulative 1029, this segment 496, len(deltas) 6 add_node add_edge commit  
loss 0.2133409693600253, batches cumulative 1663, this segment 634, len(deltas) 7 add_node add_edge commit  
loss 0.15083072343727855, batches cumulative 2405, this segment 742, len(deltas) 9 add_node add_edge commit  
loss 0.10631921172280644, batches cumulative 2750, this segment 345, len(deltas) 4 add_node add_edge commit  
loss 0.074808903787351, batches cumulative 2894, this segment 144, len(deltas) 2 add_node add_edge commit  
loss 0.0528786233468505

KeyboardInterrupt: 

In [ ]:
%debug

In [ ]:
assert False, "stop"

In [ ]:
len(deltas), deltas[0]

In [ ]:
deltas

In [ ]:
deltas[0][1]._fields

In [ ]:
list(zip(*(v[1] for v in deltas)))

In [ ]:
dict(zip(deltas[0][1]._fields, zip(*(v[1] for v in deltas))))

In [ ]:
losses, loss_steps, traj_L2_sqs, traj_cos_sq_signeds = zip(*deltas)

In [ ]:
dict(zip('losses loss_steps traj_L2_sqs traj_cos_sq_signeds'.split(), zip(*deltas)))

In [ ]:
min(traj_cos_sq_signeds), max(traj_cos_sq_signeds)

In [ ]:
100<<10

In [ ]:
len(deltas)

In [ ]:
deltas[0], deltas[-1]

In [ ]:
loss

In [ ]:
2**0.5/2

In [ ]:
net.losses([training_batch])

In [ ]:
net.learn([training_batch])

In [ ]:
net.deltas()

In [ ]:
prior_node = net_start

In [ ]:
def train_some(net, training_batch_cluster, n):
    return list((net.learn(training_batch_cluster), net.deltas())[1] for i in range(n))

In [ ]:
deltas = train_some(net, [training_batch], 3)

In [ ]:
present_node = Node(label='net',
                    properties={'shorthand': net.shorthand,
                                'ksv': tput(net.state_vector()),
                               })

In [ ]:
exp_dag.add_node(present_node)

In [ ]:
mutation = Edge(prior_node, 'trained', present_node, \
               properties={'deltas': [list(v) for v in deltas]})

In [ ]:
exp_dag.add_edge(mutation)

In [ ]:
exp_dag.commit()

In [ ]:
query = """MATCH (n:net)-[t:trained]->(nn:net)
           RETURN n.shorthand, n.ksv, t.deltas, nn.ksv"""

In [ ]:
result = exp_dag.query(query)

In [ ]:
list(as_dicts(result))

In [ ]:
assert False, "stop here"

In [ ]:
result.header

In [ ]:
result.result_set

In [ ]:
xor_net(xor_training_batch[0])

In [ ]:
sv = tget('3b8bb063e8eab703')
sv

In [ ]:
t.set_state_from_vector(sv)

In [ ]:
t.layers[0].M

In [ ]:
t2 = to_net(result)
t2

In [ ]:
redis_graph = Graph('social', r)

john = Node(label='person', properties={'name': 'John Doe', 'age': 33, 'gender': 'male', 'status': 'single'})
redis_graph.add_node(john)

japan = Node(label='country', properties={'name': 'Japan'})
redis_graph.add_node(japan)

edge = Edge(john, 'visited', japan, properties={'purpose': 'pleasure'})
redis_graph.add_edge(edge)

redis_graph.commit()

In [ ]:
query = """MATCH (p:person)-[v:visited {purpose:"pleasure"}]->(c:country)
		   RETURN p.name, p.age, v.purpose, c.name"""

result = redis_graph.query(query)

# Print resultset
result.pretty_print()

In [ ]:
# Use parameters
params = {'purpose':"pleasure"}
query = """MATCH (p:person)-[v:visited {purpose:$purpose}]->(c:country)
		   RETURN p.name, p.age, v.purpose, c.name"""

result = redis_graph.query(query, params)

# Print resultset
result.pretty_print()

In [ ]:
# Use query timeout to raise an exception if the query takes over 10 milliseconds
result = redis_graph.query(query, params, timeout=10)

In [ ]:
# Iterate through resultset
for record in result.result_set:
    person_name = record[0]
    person_age = record[1]
    visit_purpose = record[2]
    country_name = record[3]
    print(f"{person_name}, age {person_age}, visited {country_name} for {visit_purpose}")

In [ ]:
query = """MATCH p = (:person)-[:visited {purpose:"pleasure"}]->(:country) RETURN p"""

result = redis_graph.query(query)

In [ ]:
# Iterate through resultset
for record in result.result_set:
    path = record[0]
    print(path)

In [ ]:
# All done, remove graph.
redis_graph.delete()

___

In [ ]:
assert False, "stop here"

# Messing around

## Make some various 'tensors'

In [ ]:
small = np.random.randn(3)

In [ ]:
med = np.random.randn(64*64).reshape(64,64)

In [ ]:
large = np.random.randn(1024*1024).reshape(1024,1024)

In [ ]:
huge = np.random.randn(1<<(12+12)).reshape(1<<12,1<<12)

In [ ]:
mv = memoryview(huge)
mv.nbytes, mv.itemsize, mv.shape, mv.contiguous

## Put'em and get'em

In [ ]:
huge_k = tput(huge)

In [ ]:
np.equal(huge, tget(huge_k)).all()

In [ ]:
%%time
tget(huge_k)

### 

In [ ]:
%timeit tget(huge_k)

In [ ]:
%%time
con.save()

In [ ]:
t = np.array([1, 2, np.array([3, 4, np.array([5, 6]), 7]), 8])

In [ ]:
k = tput(t)

In [ ]:
%debug

In [ ]:
con.shutdown()

In [ ]:
con.tensorset('small', small)

In [ ]:
back = con.tensorget('small')

In [ ]:
np.equal(back, small).all()

In [ ]:
med_k = tput(med)

In [ ]:
med.shape, med.dtype

In [ ]:
con.tensorset('med', med)

In [ ]:
back = con.tensorget('med')

In [ ]:
np.equal(back, med).all()

In [ ]:
mv = memoryview(small)
mv.nbytes, mv.itemsize, mv.shape, mv.contiguous

In [ ]:
memoryview(huge).nbytes

In [ ]:
sha256(med).hexdigest()

In [ ]:
globals()['small']

In [ ]:
np.equal(med, tget(med_k)).all()

In [ ]:
large_k = tput(large)

In [ ]:
np.equal(large, tget(large_k)).all()

In [ ]:
huge = np.random.randn(1<<(12+12)).reshape(1<<12,1<<12)

In [ ]:
huge_k = tput(huge)

In [ ]:
np.equal(huge, tget(huge_k)).all()

In [ ]:
del(huge)

In [ ]:
import gc

In [ ]:
dir(gc)

In [ ]:
gc.get_stats()

In [ ]:
len(gc.get_objects())

In [ ]:
gc.collect()

In [ ]:
del(large)

In [ ]:
small_k = tput(small)

In [ ]:
small_k, med_k, large_k, huge_k

In [ ]:
small_k, med_k, large_k, huge_k = ('b7cd41aff50429d2',
 '10b6611fbced0d1b',
 'a02eed90c24aa100',
 '9828680612e33abc')

In [ ]:
small = tget(small_k)

In [ ]:
med = tget(med_k)

In [ ]:
large = tget(large_k)

In [ ]:
huge = tget(huge_k)

In [ ]:
huge.shape

In [ ]:
dir(con)

In [ ]:
con.info()

In [ ]:
con.config_get()

In [ ]:
con.config_set()

In [ ]:
con.config_rewrite()

In [ ]:
%timeit con.ping()

In [ ]:
con.ping()

In [ ]:
con.

In [ ]:
con.shutdown?

In [ ]:
con.shutdown()

In [ ]:
con.loadbackend('TORCH', )

In [ ]:
con.ping()

In [ ]:
con.shutdown()

In [ ]:
np.abs(np.arange(3)-1)

In [ ]:
np.multiply?